In [1]:
logB = 32
B = 2**logB

In [7]:
class FFT:
    def generator(self,n):
        return e**(-2*i*pi / (3*n));
    
    def __init__(self,n):
        self.n = n;

        fac = factor(n);
    
        if len(fac) == 2:
            self.Radix2 = fac[0][1];
            self.Radix3 = fac[1][1];

        if len(fac) == 1:
            if fac[0][0] == 2:
                self.Radix2 = fac[0][1];
                self.Radix3 = 0;

            if fac[0][0] == 3:
                self.Radix2 = 0;
                self.Radix3 = fac[0][1];

        self.level = self.Radix2+self.Radix3;

        self.w = self.generator(n);
        self.omega = self.w**n;
    
        tree = zero_matrix(ZZ,self.level+1,n);
        tree[0,0] = 3*n;

        x = self.w**0;
        x1 = round(x.real() * B)
        x2 = round(x.imag() * B)
        self.zetas = [(x1,x2)];
                
        #Radix-2 NTT with Cyclotomic Polynomial
        tree[1,0] = tree[0,0] / 6;
        tree[1,1] = 5*tree[0,0] / 6;

        x = self.w**(tree[1,0]);
        x1 = round(x.real() * B)
        x2 = round(x.imag() * B)
        self.zetas.append((x1,x2));
    
        #Radix-3 NTT
        for ll in range(1,self.Radix3+1):
            for ii in range(2*3**(ll-1)):
                tree[ll+1,3*ii  ] = tree[ll  ,  ii] / 3;
                tree[ll+1,3*ii+1] = tree[ll  ,  ii] / 3 + tree[0,0]/3;
                tree[ll+1,3*ii+2] = tree[ll  ,  ii] / 3 + 2*tree[0,0]/3;

                x = self.w**(tree[ll+1,3*ii]);
                x1 = round(x.real() * B)
                x2 = round(x.imag() * B)
                self.zetas.append((x1,x2));
                x = self.w**(tree[ll+1,3*ii]*2);
                x1 = round(x.real() * B)
                x2 = round(x.imag() * B)
                self.zetas.append((x1,x2));

        #Radix-2 NTT
        for ll in range(self.Radix3+1,self.level):
            for ii in range(2*3**(self.Radix3)*2**(ll-(self.Radix3+1))):
                tree[ll+1,2*ii  ] = tree[ll  ,  ii] / 2;
                tree[ll+1,2*ii+1] = tree[ll  ,  ii] / 2 + tree[0,0]/2;

                x = self.w**(tree[ll+1,2*ii]);
                x1 = round(x.real() * B)
                x2 = round(x.imag() * B)
                self.zetas.append((x1,x2));

        x = self.w**n;
        x1 = round(x.real() * B)
        x2 = round(x.imag() * B)
        self.omega = (x1,x2);

    def print_zetas(self):
        
        l = len(self.zetas);
        str1 = "static const complex_fp64_32 fft_zetas[%d] = {" % l
        print(str1)
        
        str = "\t";
        for ii in range(l):
            zeta1 = self.zetas[ii];
            
            str += "{%+11d, %+11d}, " % (zeta1[0], zeta1[1])
            
            if(ii % 2 == 1):
                str += "\n\t";
      
        print(str);
        print("};");
    
        print("====omega====");
        str = "static const complex_fp64_32 fft_omega = ";
        str += "{%+11d, %+11d};" % (self.omega[0], self.omega[1]);
        print(str)

    
    def fft(self, a):
        k = 1;

        b = [0] * self.n;
        for i in range(self.n):
            b[i] = (round(a[i]*B), 0);

        step = self.n/2;
        
        zeta = self.zetas[k];
        k = k+1;

        for ii in range(self.n/2):
            t = complex_mul(b[ii + step], zeta);

            b[ii + step] = (b[ii][0] - t[0] + b[ii+step][0], b[ii][1] - t[1] + b[ii+step][1]);
            b[ii       ] = (b[ii][0] + t[0],                 b[ii][1] + t[1]);

        step = self.n/6
        while step >= self.n/(6 * 3**(self.Radix3-1)):
            for start in range(0,self.n, 3*step):
                zeta1 = self.zetas[k];
                k = k + 1;
                zeta2 = self.zetas[k];
                k = k + 1;
    
                for ii in range(start,start+step):                    
                    t1 = complex_mul(b[ii +   step], zeta1);
                    t2 = complex_mul(b[ii + 2*step], zeta2);
                    t3 = (t1[0] - t2[0], t1[1] - t2[1]);
                    t3 = complex_mul(t3,self.omega);           

                    t = [0] * 6;
                    t[0] = b[ii][0] - t1[0] - t3[0];
                    t[1] = b[ii][1] - t1[1] - t3[1];
                    t[2] = b[ii][0] - t2[0] + t3[0];
                    t[3] = b[ii][1] - t2[1] + t3[1];
                    t[4] = b[ii][0] + t1[0] + t2[0];
                    t[5] = b[ii][1] + t1[1] + t2[1];
                    
                    b[ii + 2*step] = (t[0], t[1]);
                    b[ii +   step] = (t[2], t[3]);
                    b[ii         ] = (t[4], t[5]);
        
            step = step / 3;

        step = 2^(self.Radix2-2)
        while step >= 1:
            for start in range(0,self.n,(step << 1)):
                zeta = self.zetas[k];
                k += 1;

                for ii in range(start,start+step):
                    t = complex_mul(b[ii + step], zeta);
                    
                    b[ii + step] = (b[ii][0] - t[0], b[ii][1] - t[1]);
                    b[ii       ] = (b[ii][0] + t[0], b[ii][1] + t[1]);
            
            step >>= 1;
        
        return b;

In [8]:
def complex_mul(a,b):
    t1 = (a[0]*b[0] + (1 << (logB-1))) >> logB;
    t2 = (a[0]*b[1] + (1 <<(logB-1))) >> logB;
    t3 = (a[1]*b[0] + (1 <<(logB-1))) >> logB;
    t4 = (a[1]*b[1] + (1 <<(logB-1))) >> logB;

    return (t1 - t4, t2 + t3);

In [9]:
fft = FFT(768)

In [10]:
fft.print_zetas()

static const complex_fp64_32 fft_zetas[768] = {
	{+4294967296,          +0}, {+2147483648, -3719550787}, 
	{+4035949075, -1468965330}, {+3290135830, -2760751762}, 
	{ -745813244, -4229717092}, {-4035949075, +1468965330}, 
	{+4229717092,  -745813244}, {+1468965330, -4035949075}, 
	{-2760751762, -3290135830}, {+2760751762, -3290135830}, 
	{-1468965330, -4035949075}, {-4229717092,  -745813244}, 
	{+4278623649,  -374331065}, { -374331065, -4278623649}, 
	{+3518231241, -2463492036}, {-2463492036, -3518231241}, 
	{+1815131613, -3892562305}, {-3892562305, -1815131613}, 
	{+3892562305, -1815131613}, {-1815131613, -3892562305}, 
	{+2463492036, -3518231241}, {-3518231241, -2463492036}, 
	{ +374331065, -4278623649}, {-4278623649,  -374331065}, 
	{+4290879439,  -187343842}, { -187343842, -4290879439}, 
	{+2901637847, -3166582050}, {-3166582050, -2901637847}, 
	{+4096183113, -1291521575}, {-1291521575, -4096183113}, 
	{+1983195193, -3809682520}, {-3809682520, -1983195193}, 
	{+3622338677, -23076842

In [11]:
fft = FFT(1296)

In [12]:
fft.print_zetas()

static const complex_fp64_32 fft_zetas[1296] = {
	{+4294967296,          +0}, {+2147483648, -3719550787}, 
	{+4035949075, -1468965330}, {+3290135830, -2760751762}, 
	{ -745813244, -4229717092}, {-4035949075, +1468965330}, 
	{+4265926264,  -498615269}, {+4179195897,  -990487623}, 
	{+2947385392, -3124046003}, { -249730139, -4287700891}, 
	{ +249730139, -4287700891}, {-4265926264,  -498615269}, 
	{+3588392826, -2360123090}, {+1701149642, -3943708150}, 
	{+1231810505, -4114533625}, {-3588392826, -2360123090}, 
	{-1701149642, -3943708150}, {-2947385392, +3124046003}, 
	{+4291737277,  -166538951}, {+4282052077,  -332827412}, 
	{+3180612318, -2886251784}, { +415799524, -4274792958}, 
	{ +581243507, -4255455329}, {-4137646418, -1151792601}, 
	{+4137646418, -1151792601}, {+3677208833, -2219206901}, 
	{+2429263033, -3541952172}, {-1546951549, -4006705003}, 
	{ -415799524, -4274792958}, {-4214459468,  +827692857}, 
	{+3760493969, -2074952814}, {+2290095601, -3633484032}, 
	{+1546951549, -4006705